In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
image_size = [224,224]
train_loc = '../input/chest-xray-pneumonia/null'
test_loc = '../input/chest-xray-pneumonia/null'
valid_loc = '../input/chest-xray-pneumonia/null'

In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50
resnet50 = ResNet50(input_shape=image_size+[3],weights='imagenet',include_top=False)

In [4]:
resnet50.summary()

In [5]:
for layer in resnet50.layers:
    layer.trainable=False

In [6]:
from glob import glob
folders = glob('../input/chest-xray-pneumonia/chest_xray/train/*')
len(folders)

In [7]:
from tensorflow.keras.layers import Flatten
X=Flatten()(resnet50.output)

In [8]:
from tensorflow.keras.layers import Dense
pred = Dense(2,activation='softmax')(X)

In [9]:
from tensorflow.keras.models import Model
model = Model(inputs=resnet50.input,outputs=pred)
model.summary()

In [10]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dataaug = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True
                                  )

test_dataaug = ImageDataGenerator(rescale=1./255)
valid_dataaug = ImageDataGenerator(rescale=1./255)

In [12]:
train_set = train_dataaug.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/train',target_size=(224,224),batch_size=32,class_mode='categorical')

In [13]:
test_set = test_dataaug.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/test',target_size=(224,224),batch_size=32,class_mode='categorical')

In [14]:
valid_set = valid_dataaug.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/val',target_size=(224,224),batch_size=32,class_mode='categorical')

In [15]:
model.fit(train_set,validation_data=test_set,epochs=2)

In [16]:
y_pred = model.predict(test_set)
y_pred

In [17]:
import numpy as np
y_pred = np.argmax(y_pred,axis=1)
y_pred

In [18]:
from tensorflow.keras.preprocessing import image
img1 = image.load_img('../input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1437-0001.jpeg',target_size=(224,224))

In [19]:
img2 = image.load_img('../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1947_bacteria_4876.jpeg',target_size=(224,224))

In [20]:
x_norm = image.img_to_array(img1)
x_norm

In [21]:
x_inf = image.img_to_array(img2)
x_inf.shape

In [22]:
x_norm = x_norm/255
x_inf = x_inf/255

In [23]:
x_inf.shape

In [24]:
from tensorflow.keras.applications.resnet50 import preprocess_input
x_norm=np.expand_dims(x_norm,axis=0)
norm_data = preprocess_input(x_norm)

In [25]:
x_inf=np.expand_dims(x_inf,axis=0)
inf_data = preprocess_input(x_inf)
inf_data.shape

In [26]:
model.predict(norm_data)

In [27]:
model.predict(inf_data)

In [28]:
p = np.argmax(model.predict(inf_data),axis=1)
if p==1:
    print("uninfected")
else:
    print("Infected")
          